In [6]:
import pandas as pd
import requests as re
import json

from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

In [20]:
base_url = "https://gujarati.indianexpress.com/"

categories = {'gujarat':0, 'entertainment':0, 'career':0, 'sports':0,
             'dharma':0, 'news':0, 'elections':0, 'business':0,
             'lifestyle':0, 'top-news':0}

In [ ]:
try:
    with open("base_url.json", 'r') as f:
        base_url_dict = json.load(f)
except Exception as e:
    base_url_dict = {}

headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
for category in categories.keys():

    flag = 1
    page = 1
    count = 0
    while(flag):

        url = base_url+category+'/page/'+str(page)
        response = re.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        articles = soup.find_all('div', class_="ie-news-content")
        if len(articles) == 0:
            if count>=10:
                break
            page+=1
        for article in articles:

            a = article.find('a')
            base_url_dict[a['href']] = 0
        
        page+=1
        count+=1

    categories[category] = 1
    print(category)

In [29]:
with open("base_url.json", 'w') as f:
    json.dump(base_url_dict, f, indent=4)

In [ ]:
with open('./base_url.json', 'r') as f:

    base_url_dict = json.load(f)

try:
    text_df = pd.read_csv('./indianexpress_text.csv')
except FileNotFoundError:
    text_df = pd.DataFrame(columns=['Link','Text'])

for idx, url in enumerate(base_url_dict.keys()):

    if base_url_dict[url] == 1:
        continue

    text = []
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = re.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    x = soup.find_all('p')
    for p in x:
        text.append(p.get_text())
    
    text_df.loc[len(text_df),:] = [url, text]
    base_url_dict[url] = 1

    if idx%10 == 0:
        with open('./base_url.json', 'w') as f:

            json.dump(base_url_dict, f, indent=4)

        text_df.to_csv("./indianexpress_text.csv")